## Day 34 Lecture 2 Assignment

In this assignment, we will learn about gradient boosting. We will use a dataset describing TripAdvisor reviews for Las Vegas hotels loaded below and analyze the model generated for this dataset.

In [4]:
%matplotlib inline

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

import ds_useful as ds

In [5]:
vegas = pd.read_csv('https://tf-assets-prod.s3.amazonaws.com/tf-curric/data-science/LasVegasTripAdvisorReviews-Dataset.csv', sep=';')

In [6]:
vegas.head()

,User country,Nr. reviews,Nr. hotel reviews,Helpful votes,Score,Period of stay,Traveler type,Pool,Gym,Tennis court,Spa,Casino,Free internet,Hotel name,Hotel stars,Nr. rooms,User continent,Member years,Review month,Review weekday
0,USA,11,4,13,5,Dec-Feb,Friends,NO,YES,NO,NO,YES,YES,Circus Circus Hotel & Casino Las Vegas,3,3773,North America,9,January,Thursday
1,USA,119,21,75,3,Dec-Feb,Business,NO,YES,NO,NO,YES,YES,Circus Circus Hotel & Casino Las Vegas,3,3773,North America,3,January,Friday
2,USA,36,9,25,5,Mar-May,Families,NO,YES,NO,NO,YES,YES,Circus Circus Hotel & Casino Las Vegas,3,3773,North America,2,February,Saturday
3,UK,14,7,14,4,Mar-May,Friends,NO,YES,NO,NO,YES,YES,Circus Circus Hotel & Casino Las Vegas,3,3773,Europe,6,February,Friday
4,Canada,5,5,2,4,Mar-May,Solo,NO,YES,NO,NO,YES,YES,Circus Circus Hotel & Casino Las Vegas,3,3773,North America,7,March,Tuesday


Check for missing data and remove all rows with missing data

In [7]:
# answer below: 

ds.missingness_summary(vegas)

Review weekday       0.0
Review month         0.0
Nr. reviews          0.0
Nr. hotel reviews    0.0
Helpful votes        0.0
Score                0.0
Period of stay       0.0
Traveler type        0.0
Pool                 0.0
Gym                  0.0
Tennis court         0.0
Spa                  0.0
Casino               0.0
Free internet        0.0
Hotel name           0.0
Hotel stars          0.0
Nr. rooms            0.0
User continent       0.0
Member years         0.0
User country         0.0
dtype: float64

How many reviews do we have for each hotel in the dataset?

In [21]:
# answer below:

grouped = vegas.groupby(['Hotel name']).sum()
print(grouped['Nr. reviews'])

Hotel name
Bellagio Las Vegas                                      665
Caesars Palace                                          912
Circus Circus Hotel & Casino Las Vegas                  701
Encore at wynn Las Vegas                               1372
Excalibur Hotel & Casino                               1094
Hilton Grand Vacations at the Flamingo                 1162
Hilton Grand Vacations on the Boulevard                 886
Marriott's Grand Chateau                               2161
Monte Carlo Resort&Casino                              1724
Paris Las Vegas                                        1229
The Cosmopolitan Las Vegas                              811
The Cromwell                                           1717
The Palazzo Resort Hotel Casino                        1078
The Venetian Las Vegas Hotel                            769
The Westin las Vegas Hotel Casino & Spa                1115
Treasure Island- TI Hotel & Casino                      577
Tropicana Las Vegas - A Doubl

We would like to predict the score variable. Examine the dataset and decide which columns should be turned into dummy variables and transform the data. Also, where we have two columns with redundant information, remove one of the two columns. Remove the hotel stars column.

In [51]:
# answer below:

def get_categorical_columns(df, unique_thresh=10, non_binary=False):
    columns = []
    for col in df.columns:
        if df[col].nunique() < unique_thresh:
            if non_binary and df[col].nunique() < 2:
                continue
            columns.append(col)
    return columns
print(vegas.head())

# Remove repetitive Columns
remove_cols = ['Hotel stars', 'User continent', 'Period of stay']
vegas.drop(remove_cols, axis=1, inplace=True)

#Choose which columns should be one-hot-encoded
cat_cols = get_categorical_columns(vegas.drop('Score', axis=1), unique_thresh=13, non_binary=True)
dum_cols = list(vegas.select_dtypes('object').columns.drop(cat_cols)) + cat_cols

print(dum_cols)

User country  Nr. reviews  Nr. hotel reviews  Helpful votes  Score  \
0          USA           11                  4             13      5   
1          USA          119                 21             75      3   
2          USA           36                  9             25      5   
3           UK           14                  7             14      4   
4       Canada            5                  5              2      4   

  Period of stay Traveler type Pool  Gym Tennis court Spa Casino  \
0        Dec-Feb       Friends   NO  YES           NO  NO    YES   
1        Dec-Feb      Business   NO  YES           NO  NO    YES   
2        Mar-May      Families   NO  YES           NO  NO    YES   
3        Mar-May       Friends   NO  YES           NO  NO    YES   
4        Mar-May          Solo   NO  YES           NO  NO    YES   

  Free internet                              Hotel name Hotel stars  \
0           YES  Circus Circus Hotel & Casino Las Vegas           3   
1           YES  C

In [52]:
vegas_ft = pd.get_dummies(vegas, columns=dum_cols, drop_first=True)
vegas_ft

,Nr. reviews,Nr. hotel reviews,Helpful votes,Score,Nr. rooms,Member years,User country_Belgium,User country_Brazil,User country_Canada,User country_China,...,Review month_May,Review month_November,Review month_October,Review month_September,Review weekday_Monday,Review weekday_Saturday,Review weekday_Sunday,Review weekday_Thursday,Review weekday_Tuesday,Review weekday_Wednesday
0,11,4,13,5,3773,9,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
1,119,21,75,3,3773,3,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,36,9,25,5,3773,2,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
3,14,7,14,4,3773,6,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,5,5,2,4,3773,7,0,0,1,0,...,0,0,0,0,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
499,15,4,8,5,826,1,0,0,0,0,...,0,0,1,0,0,0,1,0,0,0
500,50,13,29,4,826,8,0,0,1,0,...,0,1,0,0,0,0,0,1,0,0
501,154,23,31,4,826,4,0,0,0,0,...,0,1,0,0,0,0,0,1,0,0
502,9,6,5,2,826,9,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1


Split the data into train and test (20% in test)

In [53]:
# answer below:
from sklearn.model_selection import train_test_split

X = vegas_ft.drop('Score', axis=1)
Y = vegas_ft['Score']

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=.2, random_state=101)

Create a graident boosted regression model for predicting the score. To produce the accuracy score for the test data, first round the prediction and then compare to the observed.

In [54]:
# answer below:
from sklearn.ensemble import GradientBoostingRegressor

gbr = GradientBoostingRegressor()
gbr.fit(X_train, Y_train)

def man_score(y_true, y_pred):
    residual_sum_of_squares = np.sum((y_true - y_pred)**2)
    total_sum_of_squares = np.sum((y_true - np.mean(y_true))**2)
    return 1 - (residual_sum_of_squares/total_sum_of_squares)

predictions = gbr.predict(X_test).round()

print(man_score(Y_test, predictions))
print(gbr.score(X_test, Y_test))

-0.07909701878338793
-0.04019998857797091


Try again with a learning rate of 0.8 and 0.3 and compare the results.

In [55]:
# answer below:

gbr = GradientBoostingRegressor(learning_rate=.8)
gbr.fit(X_train, Y_train)

def man_score(y_true, y_pred):
    residual_sum_of_squares = np.sum((y_true - y_pred)**2)
    total_sum_of_squares = np.sum((y_true - np.mean(y_true))**2)
    return 1 - (residual_sum_of_squares/total_sum_of_squares)

predictions = gbr.predict(X_test).round()

print(man_score(Y_test, predictions))
print(gbr.score(X_test, Y_test))

-0.8710149922453905
-0.6899405714130884


In [56]:
gbr = GradientBoostingRegressor(learning_rate=.3)
gbr.fit(X_train, Y_train)

def man_score(y_true, y_pred):
    residual_sum_of_squares = np.sum((y_true - y_pred)**2)
    total_sum_of_squares = np.sum((y_true - np.mean(y_true))**2)
    return 1 - (residual_sum_of_squares/total_sum_of_squares)

predictions = gbr.predict(X_test).round()

print(man_score(Y_test, predictions))
print(gbr.score(X_test, Y_test))

-0.12260899534723424
-0.12696698881425528
